<a href="https://colab.research.google.com/github/seismic2020/WG1-P4/blob/master/SEISMIC_AP_CLEAN_PITT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#SEISMIC AP CLEANING - Variable Conversion (PITT)
# (STILL MISSING: gpao, begin_term_cum_gpa, instructor_name, apyear)

#### Setup ####
# Load packages and data
if (!require("pacman")) install.packages("pacman")
library(pacman)
pacman::p_load("tidyverse")

# Functions and settings ####
# Use dplyr for 'select'
select <- dplyr::select

In [0]:
# Load FULL MERGED Dataframe ####
df_full <- read.csv("~/Box Sync/LSAP_LRDC/Research Projects/SEISMIC/AP/SEISMIC_AP/2191_MATH_FULL.csv")     
df_full <- df_full %>%
  filter(CAMPUS_CD == "PIT")           # TOTAL N = 670191
names(df_full)

[1] "X"                        "EMPLID_H"                
 [3] "ACADEMIC_PLAN_DESCR"      "ACADEMIC_PROGRAM_CD"     
 [5] "ACADEMIC_PROGRAM_DESCR"   "BIRTH_DT"                
 [7] "GENDER_CD"                "ETHNIC_GROUP_CD"         
 [9] "ETHNIC_AMIND_FLG"         "ETHNIC_ASIAN_FLG"        
[11] "ETHNIC_BLACK_FLG"         "ETHNIC_HISPA_FLG"        
[13] "ETHNIC_PACIF_FLG"         "ETHNIC_WHITE_FLG"        
[15] "CITIZENSHIP_STATUS_DESCR" "ATP_CD"                  
[17] "HS_NAME"                  "HS_GPA"                  
[19] "HS_CLASS_PERCENTILE"      "HOME_POSTAL"             
[21] "START_TRM_CD"             "FISCAL_YEAR"             
[23] "FIRST_TIME_FRESHMAN"      "ACADEMIC_PLAN_CD"        
[25] "ACADEMIC_SUBPLAN_CD"      "ACADEMIC_SUBPLAN_DESCR"  
[27] "SAT_HIGH_VERBAL"          "SAT_HIGH_MATH"           
[29] "SAT_HIGH_WRIT"            "ACT_HIGH_ENGL"           
[31] "ACT_HIGH_MATH"            "TOEFL_SCORE"             
[33] "LAST_SCHOOL"              "TOT_TRNSFR_CREDITS"      
[35] "NBR_OF_TRANSFERS"         "AID_YEAR"                
[37] "FATHER_GRADE_LVL"         "MOTHER_GRADE_LVL"        
[39] "NUMBER_IN_FAMILY"         "AGI"                     
[41] "USERNAME_H"               "TERM_CD"                 
[43] "COURSE_TYPE"              "CAMPUS_CD"               
[45] "SUBJECT_CD"               "CATALOG_NBR"             
[47] "CLASS_NBR"                "CLASS_TITLE"             
[49] "UNITS_TAKEN"              "GRADE_POINTS"            
[51] "COURSE_GRADE_CD"          "WRITING_FLG"             
[53] "COLLEGE_IN_HS_FLG"        "REPEAT_CD"               
[55] "ENRL_ADD_DT"              "FIRST_GENERATION_DESCR"  
[57] "CLASS_COMPONENT_DESCR"    "GRADING_BASIS_DESCR"     
[59] "NUMBER_ENROLLED"          "MEAN_GPA"                
[61] "MEDIAN_GPA"               "STAND_DEV"               
[63] "BY"                       "CH"                      
[65] "PY1"                      "PY2"                     
[67] "PHCE"                     "PHCM"                    
[69] "MAB"                      "MBC"

In [0]:
#### CLEANING ####

## STABLE - Student Level ####
df_std <- df_full %>%
  # Rename variables
  mutate(st_id = EMPLID_H) %>%
  mutate(firstgen = recode(FIRST_GENERATION_DESCR, "First Generation" = 1, "Not First Generation" = 0, "Unknown" = 0)) %>%
  mutate(ethniccode = ETHNIC_GROUP_CD) %>%
  mutate(ethniccode_cat = recode(ETHNIC_GROUP_CD, "HISPA" = 1, "BLACK" = 1, "AMIND" = 1, "PACIF" = 1, "ASIAN" = 2, "WHITE" = 0)) %>%
  mutate(urm = recode(ETHNIC_GROUP_CD, "HISPA" = 1, "BLACK" = 1, "AMIND" = 1, "PACIF" = 1, "ASIAN" = 0, "WHITE" = 0)) %>%
  mutate(gender = recode(GENDER_CD, "F"=1, "M"=0, "m"=0, "U" = 2)) %>%
  mutate(female = recode(GENDER_CD, "F"=1, "M"=0, "m"=0, "U" = 2)) %>%
  mutate(famincome = abs(AGI)) %>%
  mutate(lowincomflag = if_else(is.na(AGI), 0,
                                if_else(AGI <= 46435, 1,0))) %>%
  mutate(transfer = if_else(TOT_TRNSFR_CREDITS <= 16 | is.na(TOT_TRNSFR_CREDITS), 0, 1)) %>%
  mutate(international = if_else(CITIZENSHIP_STATUS_DESCR == "U.S. Citizen", 0, 1)) %>%
  mutate(ell = if_else(TOEFL_SCORE > 0, 0, 1)) %>%
  mutate(us_hs = if_else(is.na(HS_GPA), 0, 1)) %>%
  separate(as.character("START_TRM_CD"), c("YEAR", "SEMESTER"), 3, remove = FALSE) %>%
  separate(as.character("YEAR"), c("DEC", "YEAR"), 1) %>%
  mutate(cohort = as.numeric(YEAR)) %>%
  mutate(cohort = ifelse(cohort >= 20, cohort + 1900, cohort + 2000)) %>%
  mutate(cohort_2013 = ifelse(cohort == 2013, 1,0)) %>%
  mutate(cohort_2014 = ifelse(cohort == 2014, 1,0)) %>%
  mutate(cohort_2015 = ifelse(cohort == 2015, 1,0)) %>%
  mutate(cohort_2016 = ifelse(cohort == 2016, 1,0)) %>%
  mutate(cohort_2017 = ifelse(cohort == 2017, 1,0)) %>%
  mutate(cohort_2018 = ifelse(cohort == 2018, 1,0)) %>%
  #HACK for apyear (cohort year - 1)
  mutate(apyear = cohort - 1) %>%
  mutate(englsr = if_else(ACT_HIGH_MATH == 36, 800,
                          if_else(ACT_HIGH_MATH == 35, 780,
                                  if_else(ACT_HIGH_MATH == 34, 760,
                                          if_else(ACT_HIGH_MATH == 33, 740,
                                                  if_else(ACT_HIGH_MATH == 32,	720,
                                                          if_else(ACT_HIGH_MATH == 31,	710,
                                                                  if_else(ACT_HIGH_MATH == 30,	700,
                                                                          if_else(ACT_HIGH_MATH == 29,	680,
                                                                                  if_else(ACT_HIGH_MATH == 28,	660,
                                                                                          if_else(ACT_HIGH_MATH == 27,	640,
                                                                                                  if_else(ACT_HIGH_MATH == 26,	610,
                                                                                                          if_else(ACT_HIGH_MATH == 25,	590,
                                                                                                                  if_else(ACT_HIGH_MATH == 24,	580,
                                                                                                                          if_else(ACT_HIGH_MATH == 23,	560,
                                                                                                                                  if_else(ACT_HIGH_MATH == 22,	540,
                                                                                                                                          if_else(ACT_HIGH_MATH == 21,	530,
                                                                                                                                                  if_else(ACT_HIGH_MATH == 20,	520,
                                                                                                                                                          if_else(ACT_HIGH_MATH == 19,	510,
                                                                                                                                                                  if_else(ACT_HIGH_MATH == 18,	500,
                                                                                                                                                                          if_else(ACT_HIGH_MATH == 17,	470,                                                                                                                                                                                                if_else(ACT_HIGH_MATH == 16	, 430,
                                                                                                                                                                                                                                                                                                                                                                                                                   if_else(ACT_HIGH_MATH == 15	, 400,
                                                                                                                                                                                                                                                                                                                                                                                                                           if_else(ACT_HIGH_MATH == 14	, 360,
                                                                                                                                                                                                                                                                                                                                                                                                                                   if_else(ACT_HIGH_MATH == 13	, 330,
                                                                                                                                                                                                                                                                                                                                                                                                                                           if_else(ACT_HIGH_MATH == 12,	310,
                                                                                                                                                                                                                                                                                                                                                                                                                                                   if_else(ACT_HIGH_MATH == 11,	280,
                                                                                                                                                                                                                                                                                                                                                                                                                                                           if_else(ACT_HIGH_MATH == 10,	260, NaN )))))))))))))))))))))))))))) %>%
  mutate(englsr = ifelse(SAT_HIGH_VERBAL %in% NA, englsr, SAT_HIGH_VERBAL)) %>%
  mutate(mathsr = if_else(ACT_HIGH_MATH == 36, 800,
                          if_else(ACT_HIGH_MATH == 35, 780,
                                  if_else(ACT_HIGH_MATH == 34, 760,
                                          if_else(ACT_HIGH_MATH == 33, 740,
                                                  if_else(ACT_HIGH_MATH == 32,	720,
                                                          if_else(ACT_HIGH_MATH == 31,	710,
                                                                  if_else(ACT_HIGH_MATH == 30,	700,
                                                                          if_else(ACT_HIGH_MATH == 29,	680,
                                                                                  if_else(ACT_HIGH_MATH == 28,	660,
                                                                                          if_else(ACT_HIGH_MATH == 27,	640,
                                                                                                  if_else(ACT_HIGH_MATH == 26,	610,
                                                                                                          if_else(ACT_HIGH_MATH == 25,	590,
                                                                                                                  if_else(ACT_HIGH_MATH == 24,	580,
                                                                                                                          if_else(ACT_HIGH_MATH == 23,	560,
                                                                                                                                  if_else(ACT_HIGH_MATH == 22,	540,
                                                                                                                                          if_else(ACT_HIGH_MATH == 21,	530,
                                                                                                                                                  if_else(ACT_HIGH_MATH == 20,	520,
                                                                                                                                                          if_else(ACT_HIGH_MATH == 19,	510,
                                                                                                                                                                  if_else(ACT_HIGH_MATH == 18,	500,
                                                                                                                                                                          if_else(ACT_HIGH_MATH == 17,	470,
                                                                                                                                                                                  if_else(ACT_HIGH_MATH == 16	, 430,
                                                                                                                                                                                          if_else(ACT_HIGH_MATH == 15	, 400,
                                                                                                                                                                                                  if_else(ACT_HIGH_MATH == 14	, 360,
                                                                                                                                                                                                          if_else(ACT_HIGH_MATH == 13	, 330,
                                                                                                                                                                                                                  if_else(ACT_HIGH_MATH == 12,	310,
                                                                                                                                                                                                                          if_else(ACT_HIGH_MATH == 11,	280,
                                                                                                                                                                                                                                  if_else(ACT_HIGH_MATH == 10,	260, NaN )))))))))))))))))))))))))))) %>%
  mutate(mathsr = ifelse(SAT_HIGH_MATH %in% NA, mathsr, SAT_HIGH_MATH)) %>%
  mutate(hsgpa = HS_GPA) %>%
  # Select only common-named variables used in analysis
  select(st_id, firstgen:hsgpa) %>%
  # Remove any duplicate cases
  distinct()

names(df_std)
head(df_std)

# write.csv(df_std, file = "SEISMIC_AP_STUDENT_CLEAN.csv")          #UNIQUE N = 2297

[1] "st_id"          "firstgen"       "ethniccode"     "ethniccode_cat"
 [5] "urm"            "gender"         "female"         "famincome"     
 [9] "lowincomflag"   "transfer"       "international"  "ell"           
[13] "us_hs"          "cohort"         "cohort_2013"    "cohort_2014"   
[17] "cohort_2015"    "cohort_2016"    "cohort_2017"    "cohort_2018"   
[21] "apyear"         "englsr"         "mathsr"         "hsgpa"

,st_id,firstgen,ethniccode,ethniccode_cat,urm,gender,female,famincome,lowincomflag,transfer,⋯,cohort_2013,cohort_2014,cohort_2015,cohort_2016,cohort_2017,cohort_2018,apyear,englsr,mathsr,hsgpa
,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1343E4A4249C8EE,0,WHITE,0,0,0,0,NA,0,0,⋯,0,0,0,0,0,0,1990,NA,NA,NA
2,AE6D6CD476584F0,0,WHITE,0,0,1,1,NA,0,0,⋯,0,0,0,0,0,0,2000,NA,NA,NA
3,8D3567AF3DA6BE0,0,WHITE,0,0,0,0,NA,0,0,⋯,0,0,0,0,0,0,1997,NA,NA,NA
4,01A454AA1E3358B,0,WHITE,0,0,0,0,NA,0,0,⋯,0,0,0,0,0,0,1996,NA,NA,NA
5,36D09EC33D44582,0,ASIAN,2,0,0,0,NA,0,0,⋯,0,0,0,0,0,0,1996,630,560,NA
6,565C5D56729A5C1,0,WHITE,0,0,1,1,NA,0,1,⋯,0,0,0,0,0,0,1998,NA,NA,NA


In [0]:
#### CLEANING ####

#### Course Level ####
df_crs <- df_full %>%
  # Rename variables
  mutate(st_id = EMPLID_H) %>%
  mutate(crs_sbj = SUBJECT_CD) %>%
  mutate(crs_catalog = CATALOG_NBR) %>%
  mutate(crs_name	= CLASS_TITLE) %>%
  mutate(numgrade = GRADE_POINTS/UNITS_TAKEN) %>%
  mutate(numgrade_w = if_else(COURSE_GRADE_CD == "W", 1, 0)) %>%
  mutate(crs_retake = REPEAT_CD) %>%
  mutate(crs_term	= TERM_CD) %>%
  mutate(summer_crs = if_else(endsWith(as.character(TERM_CD),"7"), 1, 0)) %>%
  mutate(TERM_REF = START_TRM_CD-TERM_CD) %>%
  separate(as.character("START_TRM_CD"), c("YEAR", "SEMESTER"), 3, remove = FALSE) %>%
  separate(as.character("YEAR"), c("DEC", "YEAR"), 1) %>%
  mutate(enrl_from_cohort = if_else(SEMESTER == "1" & TERM_REF == "0", 1,
                                    if_else(TERM_REF == -3, 1.5,
                                            if_else(TERM_REF == -6, 1.66,
                                                    if_else(TERM_REF == -10, 2,
                                                            if_else(TERM_REF == -13, 2.5,
                                                                    if_else(TERM_REF == -16, 2.66,
                                                                            if_else(TERM_REF == -20, 3,
                                                                                    if_else(TERM_REF == -23, 3.5,
                                                                                            if_else(TERM_REF == -26, 3.66,
                                                                                                    if_else(TERM_REF == -30, 4,
                                                                                                            if_else(TERM_REF == -33, 4.5,
                                                                                                                    if_else(TERM_REF == -36, 4.66, 
                                                                                                                            if_else(SEMESTER == "4" & TERM_REF == "0", 1,
                                                                                                                                    if_else(TERM_REF == -3, 1.5,
                                                                                                                                            if_else(TERM_REF == -7, 1.66,
                                                                                                                                                    if_else(TERM_REF == -10, 2,
                                                                                                                                                            if_else(TERM_REF == -13, 2.5,
                                                                                                                                                                    if_else(TERM_REF == -17, 2.66,
                                                                                                                                                                            if_else(TERM_REF == -20, 3,
                                                                                                                                                                                    if_else(TERM_REF == -23, 3.5,
                                                                                                                                                                                            if_else(TERM_REF == -27, 3.66,
                                                                                                                                                                                                    if_else(TERM_REF == -30, 4,
                                                                                                                                                                                                            if_else(TERM_REF == -33, 4.5,
                                                                                                                                                                                                                    if_else(TERM_REF == -37, 4.66, 
                                                                                                                                                                                                                            if_else(SEMESTER == "7" & TERM_REF == "0", 1,
                                                                                                                                                                                                                                    if_else(TERM_REF == -4, 1.5,
                                                                                                                                                                                                                                            if_else(TERM_REF == -7, 1.66,
                                                                                                                                                                                                                                                    if_else(TERM_REF == -10, 2,
                                                                                                                                                                                                                                                            if_else(TERM_REF == -14, 2.5,
                                                                                                                                                                                                                                                                    if_else(TERM_REF == -17, 2.66,
                                                                                                                                                                                                                                                                            if_else(TERM_REF == -20, 3,
                                                                                                                                                                                                                                                                                    if_else(TERM_REF == -24, 3.5,
                                                                                                                                                                                                                                                                                            if_else(TERM_REF == -26, 3.66,
                                                                                                                                                                                                                                                                                                    if_else(TERM_REF == -30, 4,
                                                                                                                                                                                                                                                                                                            if_else(TERM_REF == -34, 4.5,
                                                                                                                                                                                                                                                                                                                    if_else(TERM_REF == -36, 4.66, NaN ))))))))))))))))))))))))))))))))))))) %>%
  #mutate(gpao = ?) %>%
  #mutate(begin_term_cum_gpa = ?) %>%
  mutate(crs_credits = UNITS_TAKEN) %>%
  #mutate(instructor_name = NA) %>%
  mutate(crs_component= CLASS_COMPONENT_DESCR) %>%
  mutate(class_number	= CLASS_NBR) %>%
  mutate(current_major = ACADEMIC_SUBPLAN_DESCR) %>%
  # Select only common-named variables used in analysis
  select(st_id, crs_sbj:current_major)

names(df_crs)
head(df_crs)

# write.csv(df_crs, file = "SEISMIC_AP_COURSES_CLEAN.csv")


[1] "st_id"            "crs_sbj"          "crs_catalog"      "crs_name"        
 [5] "numgrade"         "numgrade_w"       "crs_retake"       "crs_term"        
 [9] "summer_crs"       "TERM_REF"         "enrl_from_cohort" "crs_credits"     
[13] "crs_component"    "class_number"     "current_major"

,st_id,crs_sbj,crs_catalog,crs_name,numgrade,numgrade_w,crs_retake,crs_term,summer_crs,TERM_REF,enrl_from_cohort,crs_credits,crs_component,class_number,current_major
,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<int>,<dbl>,<int>,<dbl>,<dbl>,<fct>,<int>,<fct>
1,1343E4A4249C8EE,CS,0441,DISCRETE STRUCTURES FOR CS,4.00,0,N,2171,0,-260,NaN,3,Lecture,30436,No Subplan
2,1343E4A4249C8EE,STAT,1000,APPLIED STATISTICAL METHODS,4.00,0,N,2174,0,-263,NaN,4,Lecture,31103,No Subplan
3,1343E4A4249C8EE,STAT,1361,STATSTCL LEARNING & DATA SCI,3.25,0,N,2184,0,-273,NaN,3,Lecture,31123,No Subplan
4,1343E4A4249C8EE,MATH,0240,ANALYTC GEOMETRY & CALCULUS 3,4.00,0,N,2187,1,-276,NaN,4,Lecture,10094,No Subplan
5,1343E4A4249C8EE,HPS,1616,ARTFCL INTELGNC & PHIL OF SCI,4.00,0,N,2191,0,-280,NaN,3,Lecture,27604,No Subplan
6,1343E4A4249C8EE,STAT,1631,INTERMEDIATE PROBABILITY,3.25,0,N,2191,0,-280,NaN,3,Lecture,10536,No Subplan


In [0]:
#### Course Level ####
# Subset By Course (Taking only First Attempt) ####
# Bio
df_crs_bio1 <- df_crs %>%
  filter(crs_sbj == "BIOSC" & (crs_catalog == "0150")) %>% # | crs_catalog == "0715")) %>%
  #Only first time taking course
  group_by(st_id, crs_catalog) %>% 
  arrange(crs_term, .by_group= TRUE) %>%
  mutate(crs_retake_num = row_number()) %>%
  filter(crs_retake_num == 1) %>%
  #Only first time taking course (incl. Honors)
  group_by(st_id) %>% 
  arrange(crs_term, .by_group= TRUE) %>%
  mutate(crs_retake_num2 = row_number()) %>%
  filter(crs_retake_num2 == 1)

head(df_crs_bio1)

df_crs_bio2 <- df_crs %>%
  filter(crs_sbj == "BIOSC" & (crs_catalog == "0160")) %>% # | crs_catalog == "0716")) %>%
  #Only first time taking course
  group_by(st_id, crs_catalog) %>% 
  arrange(crs_term, .by_group= TRUE) %>%
  mutate(crs_retake_num = row_number()) %>%
  filter(crs_retake_num == 1) %>%
  #Only first time taking course (incl. Honors)
  group_by(st_id) %>% 
  arrange(crs_term, .by_group= TRUE) %>%
  mutate(crs_retake_num2 = row_number()) %>%
  filter(crs_retake_num2 == 1)

head(df_crs_bio2)

#Chem
df_crs_chem1 <- df_crs %>%
  filter(crs_sbj == "CHEM" & (crs_catalog == "0110")) %>% # | crs_catalog == "0710")) %>%
  #Only first time taking course
  group_by(st_id, crs_catalog) %>% 
  arrange(crs_term, .by_group= TRUE) %>%
  mutate(crs_retake_num = row_number()) %>%
  filter(crs_retake_num == 1) %>%
  #Only first time taking course (incl. Honors)
  group_by(st_id) %>% 
  arrange(crs_term, .by_group= TRUE) %>%
  mutate(crs_retake_num2 = row_number()) %>%
  filter(crs_retake_num2 == 1)

head(df_crs_chem1)

df_crs_chem2 <- df_crs %>%
  filter(crs_sbj == "CHEM" & (crs_catalog == "0120")) %>% # | crs_catalog == "0720")) %>%
  #Only first time taking course
  group_by(st_id, crs_catalog) %>% 
  arrange(crs_term, .by_group= TRUE) %>%
  mutate(crs_retake_num = row_number()) %>%
  filter(crs_retake_num == 1) %>%
  #Only first time taking course (incl. Honors)
  group_by(st_id) %>% 
  arrange(crs_term, .by_group= TRUE) %>%
  mutate(crs_retake_num2 = row_number()) %>%
  filter(crs_retake_num2 == 1)

head(df_crs_chem2)

#Phys
df_crs_phys1 <- df_crs %>%
  filter(crs_sbj == "PHYS" & (crs_catalog == "0174")) %>% # | crs_catalog == "0475")) %>%
  #Only first time taking course
  group_by(st_id, crs_catalog) %>% 
  arrange(crs_term, .by_group= TRUE) %>%
  mutate(crs_retake_num = row_number()) %>%
  filter(crs_retake_num == 1) %>%
  #Only first time taking course (incl. Honors)
  group_by(st_id) %>% 
  arrange(crs_term, .by_group= TRUE) %>%
  mutate(crs_retake_num2 = row_number()) %>%
  filter(crs_retake_num2 == 1)

head(df_crs_phys1)

df_crs_phys2 <- df_crs %>%
  filter(crs_sbj == "PHYS" & (crs_catalog == "0175")) %>% # | crs_catalog == "0476")) %>%
  #Only first time taking course
  group_by(st_id, crs_catalog) %>% 
  arrange(crs_term, .by_group= TRUE) %>%
  mutate(crs_retake_num = row_number()) %>%
  filter(crs_retake_num == 1) %>%
  #Only first time taking course (incl. Honors)
  group_by(st_id) %>% 
  arrange(crs_term, .by_group= TRUE) %>%
  mutate(crs_retake_num2 = row_number()) %>%
  filter(crs_retake_num2 == 1)

head(df_crs_phys2)

st_id,crs_sbj,crs_catalog,crs_name,numgrade,numgrade_w,crs_retake,crs_term,summer_crs,TERM_REF,enrl_from_cohort,crs_credits,crs_component,class_number,current_major,crs_retake_num,crs_retake_num2
<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<int>,<dbl>,<int>,<dbl>,<dbl>,<fct>,<int>,<fct>,<int>,<int>
0036734B93AA439,BIOSC,0150,FOUNDATIONS OF BIOLOGY 1,0.00,0,R,2131,0,0,1.0,3,Lecture,10249,Biological Sciences,1,1
003E74BA1CCF4D4,BIOSC,0150,FOUNDATIONS OF BIOLOGY 1,3.00,0,N,2101,0,-10,2.0,3,Lecture,10546,Pre-Occupational Therapy,1,1
0042D682C64F6A3,BIOSC,0150,FOUNDATIONS OF BIOLOGY 1,2.00,0,N,2121,0,-20,3.0,3,Lecture,11094,Neuroscience,1,1
0042D86A430B931,BIOSC,0150,FOUNDATIONS OF BIOLOGY 1,3.25,0,N,2071,0,0,1.0,3,Lecture,11289,Anthropology,1,1
004325A54F42C34,BIOSC,0150,FOUNDATIONS OF BIOLOGY 1,3.25,0,N,2164,0,-13,2.5,3,Lecture,18514,Pre-Physical Therapy,1,1
0043C31694C07F6,BIOSC,0150,FOUNDATIONS OF BIOLOGY 1,1.00,0,R,2121,0,0,1.0,3,Lecture,10258,Undeclared,1,1


st_id,crs_sbj,crs_catalog,crs_name,numgrade,numgrade_w,crs_retake,crs_term,summer_crs,TERM_REF,enrl_from_cohort,crs_credits,crs_component,class_number,current_major,crs_retake_num,crs_retake_num2
<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<int>,<dbl>,<int>,<dbl>,<dbl>,<fct>,<int>,<fct>,<int>,<int>
00351BABB5EE161,BIOSC,0160,FOUNDATIONS OF BIOLOGY 2,3.25,0,N,2171,0,0,1.0,3,Lecture,20455,Pre-Medicine,1,1
003E74BA1CCF4D4,BIOSC,0160,FOUNDATIONS OF BIOLOGY 2,2.00,0,N,2104,0,-13,2.5,3,Lecture,12250,Pre-Occupational Therapy,1,1
0043C31694C07F6,BIOSC,0160,FOUNDATIONS OF BIOLOGY 2,2.25,0,N,2131,0,-10,2.0,3,Lecture,10664,Undeclared,1,1
00476F70B4C3000,BIOSC,0160,FOUNDATIONS OF BIOLOGY 2,3.00,0,N,2104,0,-13,2.5,3,Lecture,12250,Pre-Dentistry,1,1
004D0D22BF5B764,BIOSC,0160,FOUNDATIONS OF BIOLOGY 2,0.00,0,N,2194,0,-3,1.5,3,Lecture,10772,Biological Sciences,1,1
006193FE8B693CE,BIOSC,0160,FOUNDATIONS OF BIOLOGY 2,4.00,0,N,2164,0,-3,1.5,3,Lecture,10924,Biological Sciences,1,1


st_id,crs_sbj,crs_catalog,crs_name,numgrade,numgrade_w,crs_retake,crs_term,summer_crs,TERM_REF,enrl_from_cohort,crs_credits,crs_component,class_number,current_major,crs_retake_num,crs_retake_num2
<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<int>,<dbl>,<int>,<dbl>,<dbl>,<fct>,<int>,<fct>,<int>,<int>
001D1DE2181ADA8,CHEM,0110,GENERAL CHEMISTRY 1,4.00,0,N,2131,0,0,1.0,4,Lecture,10667,Pre-Medicine,1,1
0042D682C64F6A3,CHEM,0110,GENERAL CHEMISTRY 1,2.00,0,N,2101,0,0,1.0,4,Lecture,11446,Neuroscience,1,1
0042D86A430B931,CHEM,0110,GENERAL CHEMISTRY 1,1.75,0,N,2074,0,-3,1.5,4,Lecture,10753,Anthropology,1,1
004325A54F42C34,CHEM,0110,GENERAL CHEMISTRY 1,3.00,0,N,2151,0,0,1.0,4,Lecture,20454,Pre-Physical Therapy,1,1
0043C31694C07F6,CHEM,0110,GENERAL CHEMISTRY 1,1.00,0,R,2121,0,0,1.0,4,Lecture,10688,Undeclared,1,1
0044041C28D3A81,CHEM,0110,GENERAL CHEMISTRY 1,3.75,0,N,2171,0,0,1.0,4,Lecture,10550,No Subplan,1,1


st_id,crs_sbj,crs_catalog,crs_name,numgrade,numgrade_w,crs_retake,crs_term,summer_crs,TERM_REF,enrl_from_cohort,crs_credits,crs_component,class_number,current_major,crs_retake_num,crs_retake_num2
<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<int>,<dbl>,<int>,<dbl>,<dbl>,<fct>,<int>,<fct>,<int>,<int>
001D1DE2181ADA8,CHEM,0120,GENERAL CHEMISTRY 2,4.00,0,N,2134,0,-3,1.5,4,Lecture,10330,Pre-Medicine,1,1
00351BABB5EE161,CHEM,0120,GENERAL CHEMISTRY 2,3.00,0,N,2171,0,0,1.0,4,Lecture,10779,Pre-Medicine,1,1
0042D86A430B931,CHEM,0120,GENERAL CHEMISTRY 2,0.00,0,N,2084,0,-13,2.5,4,Lecture,10376,Anthropology,1,1
004325A54F42C34,CHEM,0120,GENERAL CHEMISTRY 2,3.00,0,N,2154,0,-3,1.5,4,Lecture,10311,Pre-Physical Therapy,1,1
0043C31694C07F6,CHEM,0120,GENERAL CHEMISTRY 2,1.75,0,R,2131,0,-10,2.0,4,Lecture,10939,Undeclared,1,1
0044041C28D3A81,CHEM,0120,GENERAL CHEMISTRY 2,4.00,0,N,2174,0,-3,1.5,4,Lecture,10286,No Subplan,1,1


st_id,crs_sbj,crs_catalog,crs_name,numgrade,numgrade_w,crs_retake,crs_term,summer_crs,TERM_REF,enrl_from_cohort,crs_credits,crs_component,class_number,current_major,crs_retake_num,crs_retake_num2
<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<int>,<dbl>,<int>,<dbl>,<dbl>,<fct>,<int>,<fct>,<int>,<int>
000056410B83915,PHYS,0174,BASC PHYS SCI & ENGR 1 (INTGD),3.25,0,N,2181,0,0,1.00,4,Lecture,11436,Undeclared,1,1
0005514BDBA8B3C,PHYS,0174,BASC PHYS SCI & ENGR 1 (INTGD),4.00,0,N,2151,0,0,1.00,4,Lecture,11606,Undeclared,1,1
000E987A7E3328E,PHYS,0174,BASC PHYS SCI & ENGR 1 (INTGD),2.75,0,N,2171,0,-17,2.66,4,Lecture,11243,No Subplan,1,1
0019BE913D37FC1,PHYS,0174,BASC PHYS SCI & ENGR 1 (INTGD),0.00,1,N,2151,0,0,1.00,4,Lecture,11327,Undeclared,1,1
001F8A15F93B074,PHYS,0174,BASC PHYS SCI & ENGR 1 (INTGD),0.00,1,N,2124,0,0,1.00,4,Lecture,11458,Chemical Engineering,1,1
004325A54F42C34,PHYS,0174,BASC PHYS SCI & ENGR 1 (INTGD),3.00,0,N,2164,0,-13,2.50,4,Lecture,11259,Pre-Physical Therapy,1,1


st_id,crs_sbj,crs_catalog,crs_name,numgrade,numgrade_w,crs_retake,crs_term,summer_crs,TERM_REF,enrl_from_cohort,crs_credits,crs_component,class_number,current_major,crs_retake_num,crs_retake_num2
<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<int>,<dbl>,<int>,<dbl>,<dbl>,<fct>,<int>,<fct>,<int>,<int>
000056410B83915,PHYS,0175,BASC PHYS SCI & ENGR 2 (INTGD),3.75,0,N,2184,0,-3,1.50,4,Lecture,26072,Undeclared,1,1
0005514BDBA8B3C,PHYS,0175,BASC PHYS SCI & ENGR 2 (INTGD),4.00,0,N,2154,0,-3,1.50,4,Lecture,11316,Undeclared,1,1
000E987A7E3328E,PHYS,0175,BASC PHYS SCI & ENGR 2 (INTGD),3.00,0,N,2174,0,-20,3.00,4,Lecture,27825,No Subplan,1,1
004325A54F42C34,PHYS,0175,BASC PHYS SCI & ENGR 2 (INTGD),2.25,0,N,2167,1,-16,2.66,4,Lecture,10655,Pre-Physical Therapy,1,1
0049F8A318A0EFD,PHYS,0175,BASC PHYS SCI & ENGR 2 (INTGD),2.25,0,N,2164,0,-3,1.50,4,Lecture,11120,Undeclared,1,1
005A84B762FEEBE,PHYS,0175,BASC PHYS SCI & ENGR 2 (INTGD),0.00,0,N,2194,0,-3,1.50,4,Lecture,25383,Industrial Engineering,1,1


In [0]:
#### CLEANING ####

##### AP Level ####
# Subset By course ####
# Bio
df_ap_bio <- df_full %>%
  mutate(st_id = EMPLID_H) %>%
  mutate(aptaker = ifelse(is.na(BY), 0, 1)) %>%
  mutate(apskipper = ifelse(BY >= 4 & !is.na(BY), 1, 0)) %>%
  mutate(apskipper_2 = ifelse(BY == 5 & !is.na(BY), 1, 0)) %>%
  mutate(tookcourse = ifelse(
    SUBJECT_CD == "BIOSC" & (CATALOG_NBR == "0150") & # | CATALOG_NBR == "0715") & 
      COURSE_GRADE_CD != "W", 1, 0)) %>%
  mutate(tookcourse_2 = ifelse(
    SUBJECT_CD == "BIOSC" & (CATALOG_NBR == "0160") & # | CATALOG_NBR == "0716") & 
      COURSE_GRADE_CD != "W", 1, 0)) %>%
  #mutate(apyear = ?) %>%
  mutate(apscore = as.character(BY)) %>%
  mutate(apscore_full = ifelse(is.na(BY), 0, BY)) %>%
  select(st_id, aptaker:apscore_full) %>%
  group_by(st_id) %>%
  summarize_at(vars(-group_cols()), max)

names(df_ap_bio)
head(df_ap_bio)

#Chem
df_ap_chem <- df_full %>%
  mutate(st_id = EMPLID_H) %>%
  mutate(aptaker = ifelse(is.na(CH), 0, 1)) %>%
  mutate(apskipper = ifelse(CH >= 3 & !is.na(CH), 1, 0)) %>%
  mutate(apskipper_2 = ifelse(CH == 5 & !is.na(CH), 1, 0)) %>%
  mutate(tookcourse = ifelse(
    SUBJECT_CD == "CHEM" & (CATALOG_NBR == "0110") & # | CATALOG_NBR == "0710") & 
      COURSE_GRADE_CD != "W", 1, 0)) %>%
  mutate(tookcourse_2 = ifelse(
    SUBJECT_CD == "CHEM" & (CATALOG_NBR == "0120") & # | CATALOG_NBR == "0720") & 
      COURSE_GRADE_CD != "W", 1, 0)) %>%
  #mutate(apyear = ?) %>%
  mutate(apscore = as.character(CH)) %>%
  mutate(apscore_full = ifelse(is.na(CH), 0, CH)) %>%
  select(st_id, aptaker:apscore_full) %>%
  group_by(st_id) %>%
  summarize_at(vars(-group_cols()), max)

names(df_ap_chem)
head(df_ap_chem)

#Phys
df_ap_phys <- df_full %>%
  mutate(st_id = EMPLID_H) %>%
  mutate(aptaker_CE = ifelse(is.na(PHCE), 0, 1)) %>%
  mutate(aptaker_CM = ifelse(is.na(PHCM), 0, 1)) %>%
  mutate(aptaker = ifelse(aptaker_CE == 1 | aptaker_CM == 1, 1, 0)) %>%
  mutate(apskipperCE = ifelse(PHCE == 5 & !is.na(PHCE), 1, 0)) %>%
  mutate(apskipperCM = ifelse(PHCM == 5  & !is.na(PHCM), 1, 0)) %>%
  mutate(apskipper = ifelse(apskipperCE == 1 | apskipperCM == 1, 1, 0)) %>%
  mutate(tookcourse = ifelse(
    SUBJECT_CD == "PHYS" & (CATALOG_NBR == "0174") & # | CATALOG_NBR == "0475") & 
      COURSE_GRADE_CD != "W", 1, 0)) %>%
  mutate(tookcourse_2 = ifelse(
    SUBJECT_CD == "PHYS" & (CATALOG_NBR == "0175") & # | CATALOG_NBR == "0476") & 
      COURSE_GRADE_CD != "W", 1, 0)) %>%
  #mutate(apyear = ?) %>%
  mutate(apscore_CE = PHCE) %>%
  mutate(apscore_CM = PHCM) %>%
  mutate(apscore = ifelse(is.na(PHCM), PHCE, PHCM)) %>%
  mutate(apscore = as.character(apscore)) %>%
  mutate(apscore_CE_full = ifelse(is.na(PHCE), 0, PHCE)) %>%
  mutate(apscore_CM_full = ifelse(is.na(PHCM), 0, PHCM)) %>%
  mutate(apscore_full = ifelse(is.na(PHCM), PHCE, PHCM)) %>%
  mutate(apscore_full = ifelse(is.na(apscore_full), 0, apscore_full)) %>%
  select(st_id, aptaker:apscore_full) %>%
  group_by(st_id) %>%
  summarize_at(vars(-group_cols()), max)

names(df_ap_phys)
head(df_ap_phys)

[1] "st_id"        "aptaker"      "apskipper"    "apskipper_2"  "tookcourse"  
[6] "tookcourse_2" "apscore"      "apscore_full"

st_id,aptaker,apskipper,apskipper_2,tookcourse,tookcourse_2,apscore,apscore_full
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
000056410B83915,0,0,0,0,0,NA,0
0005514BDBA8B3C,0,0,0,0,0,NA,0
00094389A4C8D52,0,0,0,0,0,NA,0
000E6BF8EF10B8F,0,0,0,0,0,NA,0
000E987A7E3328E,0,0,0,0,0,NA,0
0019BE913D37FC1,0,0,0,0,0,NA,0


[1] "st_id"        "aptaker"      "apskipper"    "apskipper_2"  "tookcourse"  
[6] "tookcourse_2" "apscore"      "apscore_full"

st_id,aptaker,apskipper,apskipper_2,tookcourse,tookcourse_2,apscore,apscore_full
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
000056410B83915,1,1,0,0,0,4,4
0005514BDBA8B3C,1,1,1,0,0,5,5
00094389A4C8D52,0,0,0,0,0,NA,0
000E6BF8EF10B8F,0,0,0,0,0,NA,0
000E987A7E3328E,0,0,0,0,0,NA,0
0019BE913D37FC1,0,0,0,0,0,NA,0


[1] "st_id"           "aptaker"         "apskipperCE"     "apskipperCM"    
 [5] "apskipper"       "tookcourse"      "tookcourse_2"    "apscore_CE"     
 [9] "apscore_CM"      "apscore"         "apscore_CE_full" "apscore_CM_full"
[13] "apscore_full"

st_id,aptaker,apskipperCE,apskipperCM,apskipper,tookcourse,tookcourse_2,apscore_CE,apscore_CM,apscore,apscore_CE_full,apscore_CM_full,apscore_full
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
000056410B83915,1,0,1,1,1,1,4,5,5,4,5,5
0005514BDBA8B3C,0,0,0,0,1,1,NA,NA,NA,0,0,0
00094389A4C8D52,0,0,0,0,0,0,NA,NA,NA,0,0,0
000E6BF8EF10B8F,0,0,0,0,0,0,NA,NA,NA,0,0,0
000E987A7E3328E,0,0,0,0,1,1,NA,NA,NA,0,0,0
0019BE913D37FC1,0,0,0,0,0,0,NA,NA,NA,0,0,0


In [0]:
#### Inclusion/Exclusion criteria #### 

# Bio (N=3090)
df_bio <- df_std %>%
  right_join(df_crs_bio2, by = "st_id") %>%
  full_join(df_crs_bio1, by = "st_id") %>%
  full_join(df_ap_bio, by = "st_id") %>%
  mutate(course = "BIO") %>%
  mutate(skipped_1 = ifelse(tookcourse == 0 & tookcourse_2 == 1, 1, 0)) %>%
  select(course, st_id:hsgpa, crs_sbj.x:current_major.x, crs_sbj.y:current_major.y, 
         aptaker, apscore, apscore_full, apskipper, 
         tookcourse, tookcourse_2, skipped_1) %>%
  filter(transfer == 0) %>%
  filter(international == 0) %>%
  filter(crs_term.x > 2137 & crs_term.x < 2191)

names(df_bio)
head(df_bio)

#Chem (N=3019)
df_chem <- df_std %>%
  right_join(df_crs_chem2, by = "st_id") %>%
  full_join(df_crs_chem1, by = "st_id") %>%
  full_join(df_ap_chem, by = "st_id") %>%
  mutate(course = "CHEM") %>%
  mutate(skipped_1 = ifelse(tookcourse == 0 & tookcourse_2 == 1, 1, 0)) %>%
  select(course, st_id:hsgpa, crs_sbj.x:current_major.x, crs_sbj.y:current_major.y, 
         aptaker, apscore, apscore_full, apskipper, 
         tookcourse, tookcourse_2, skipped_1) %>%
  filter(transfer == 0) %>%
  filter(international == 0) %>%
  filter(crs_term.x > 2147 & crs_term.x < 2191)

names(df_chem)
head(df_chem)

#Phys (N=1598)
df_phys <- df_std %>%
  right_join(df_crs_phys2, by = "st_id") %>%
  full_join(df_crs_phys1, by = "st_id") %>%
  full_join(df_ap_phys, by = "st_id") %>%
  mutate(course = "PHYS") %>%
  mutate(skipped_1 = ifelse(tookcourse == 0 & tookcourse_2 == 1, 1, 0)) %>%
  select(course, st_id:hsgpa, crs_sbj.x:current_major.x, crs_sbj.y:current_major.y, 
         aptaker, apscore, apscore_full, apskipper,
         tookcourse, tookcourse_2, skipped_1) %>%
  filter(transfer == 0) %>%
  filter(international == 0) %>%
  filter(crs_term.x > 2157 & crs_term.x < 2191)

names(df_phys)
head(df_phys)

[1] "course"             "st_id"              "firstgen"          
 [4] "ethniccode"         "ethniccode_cat"     "urm"               
 [7] "gender"             "female"             "famincome"         
[10] "lowincomflag"       "transfer"           "international"     
[13] "ell"                "us_hs"              "cohort"            
[16] "cohort_2013"        "cohort_2014"        "cohort_2015"       
[19] "cohort_2016"        "cohort_2017"        "cohort_2018"       
[22] "apyear"             "englsr"             "mathsr"            
[25] "hsgpa"              "crs_sbj.x"          "crs_catalog.x"     
[28] "crs_name.x"         "numgrade.x"         "numgrade_w.x"      
[31] "crs_retake.x"       "crs_term.x"         "summer_crs.x"      
[34] "TERM_REF.x"         "enrl_from_cohort.x" "crs_credits.x"     
[37] "crs_component.x"    "class_number.x"     "current_major.x"   
[40] "crs_sbj.y"          "crs_catalog.y"      "crs_name.y"        
[43] "numgrade.y"         "numgrade_w.y"       "crs_retake.y"      
[46] "crs_term.y"         "summer_crs.y"       "TERM_REF.y"        
[49] "enrl_from_cohort.y" "crs_credits.y"      "crs_component.y"   
[52] "class_number.y"     "current_major.y"    "aptaker"           
[55] "apscore"            "apscore_full"       "apskipper"         
[58] "tookcourse"         "tookcourse_2"       "skipped_1"

,course,st_id,firstgen,ethniccode,ethniccode_cat,urm,gender,female,famincome,lowincomflag,⋯,crs_component.y,class_number.y,current_major.y,aptaker,apscore,apscore_full,apskipper,tookcourse,tookcourse_2,skipped_1
,<chr>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,⋯,<fct>,<int>,<fct>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,BIO,006193FE8B693CE,0,WHITE,0,0,0,0,NA,0,⋯,Lecture,10889,Biological Sciences,0,NA,0,0,1,1,0
2,BIO,00767AA686DED2D,0,WHITE,0,0,0,0,153056,0,⋯,Lecture,29164,Pre-Medicine,0,NA,0,0,1,1,0
3,BIO,00806194870DD09,0,WHITE,0,0,0,0,84630,0,⋯,Lecture,10197,Biological Sciences,0,NA,0,0,1,1,0
4,BIO,008593BB28EA023,1,ASIAN,2,0,0,0,21782,1,⋯,Lecture,10199,Pharmacy,1,3,3,0,1,1,0
5,BIO,00915F37523B8A2,1,WHITE,0,0,1,1,107815,0,⋯,Lecture,29164,Biological Sciences,1,2,2,0,1,1,0
6,BIO,00A397685C01F09,0,WHITE,0,0,1,1,105930,0,⋯,Lecture,10195,Pre-Medicine,1,3,3,0,1,1,0


[1] "course"             "st_id"              "firstgen"          
 [4] "ethniccode"         "ethniccode_cat"     "urm"               
 [7] "gender"             "female"             "famincome"         
[10] "lowincomflag"       "transfer"           "international"     
[13] "ell"                "us_hs"              "cohort"            
[16] "cohort_2013"        "cohort_2014"        "cohort_2015"       
[19] "cohort_2016"        "cohort_2017"        "cohort_2018"       
[22] "apyear"             "englsr"             "mathsr"            
[25] "hsgpa"              "crs_sbj.x"          "crs_catalog.x"     
[28] "crs_name.x"         "numgrade.x"         "numgrade_w.x"      
[31] "crs_retake.x"       "crs_term.x"         "summer_crs.x"      
[34] "TERM_REF.x"         "enrl_from_cohort.x" "crs_credits.x"     
[37] "crs_component.x"    "class_number.x"     "current_major.x"   
[40] "crs_sbj.y"          "crs_catalog.y"      "crs_name.y"        
[43] "numgrade.y"         "numgrade_w.y"       "crs_retake.y"      
[46] "crs_term.y"         "summer_crs.y"       "TERM_REF.y"        
[49] "enrl_from_cohort.y" "crs_credits.y"      "crs_component.y"   
[52] "class_number.y"     "current_major.y"    "aptaker"           
[55] "apscore"            "apscore_full"       "apskipper"         
[58] "tookcourse"         "tookcourse_2"       "skipped_1"

,course,st_id,firstgen,ethniccode,ethniccode_cat,urm,gender,female,famincome,lowincomflag,⋯,crs_component.y,class_number.y,current_major.y,aptaker,apscore,apscore_full,apskipper,tookcourse,tookcourse_2,skipped_1
,<chr>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,⋯,<fct>,<int>,<fct>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,CHEM,004325A54F42C34,0,WHITE,0,0,0,0,113456,0,⋯,Lecture,20454,Pre-Physical Therapy,1,2,2,0,1,1,0
2,CHEM,0044041C28D3A81,0,ASIAN,2,0,0,0,0,1,⋯,Lecture,10550,No Subplan,0,NA,0,0,1,1,0
3,CHEM,006193FE8B693CE,0,WHITE,0,0,0,0,NA,0,⋯,Lecture,23748,Biological Sciences,1,4,4,1,1,1,0
4,CHEM,006A6C2DFF6F9DC,0,WHITE,0,0,1,1,55508,0,⋯,NA,NA,NA,1,4,4,1,0,1,1
5,CHEM,00767AA686DED2D,0,WHITE,0,0,0,0,153056,0,⋯,Lecture,10538,Pre-Medicine,1,3,3,1,1,1,0
6,CHEM,00806194870DD09,0,WHITE,0,0,0,0,84630,0,⋯,Lecture,10552,Biological Sciences,0,NA,0,0,1,1,0


[1] "course"             "st_id"              "firstgen"          
 [4] "ethniccode"         "ethniccode_cat"     "urm"               
 [7] "gender"             "female"             "famincome"         
[10] "lowincomflag"       "transfer"           "international"     
[13] "ell"                "us_hs"              "cohort"            
[16] "cohort_2013"        "cohort_2014"        "cohort_2015"       
[19] "cohort_2016"        "cohort_2017"        "cohort_2018"       
[22] "apyear"             "englsr"             "mathsr"            
[25] "hsgpa"              "crs_sbj.x"          "crs_catalog.x"     
[28] "crs_name.x"         "numgrade.x"         "numgrade_w.x"      
[31] "crs_retake.x"       "crs_term.x"         "summer_crs.x"      
[34] "TERM_REF.x"         "enrl_from_cohort.x" "crs_credits.x"     
[37] "crs_component.x"    "class_number.x"     "current_major.x"   
[40] "crs_sbj.y"          "crs_catalog.y"      "crs_name.y"        
[43] "numgrade.y"         "numgrade_w.y"       "crs_retake.y"      
[46] "crs_term.y"         "summer_crs.y"       "TERM_REF.y"        
[49] "enrl_from_cohort.y" "crs_credits.y"      "crs_component.y"   
[52] "class_number.y"     "current_major.y"    "aptaker"           
[55] "apscore"            "apscore_full"       "apskipper"         
[58] "tookcourse"         "tookcourse_2"       "skipped_1"

,course,st_id,firstgen,ethniccode,ethniccode_cat,urm,gender,female,famincome,lowincomflag,⋯,crs_component.y,class_number.y,current_major.y,aptaker,apscore,apscore_full,apskipper,tookcourse,tookcourse_2,skipped_1
,<chr>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,⋯,<fct>,<int>,<fct>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,PHYS,000056410B83915,0,WHITE,0,0,0,0,162998,0,⋯,Lecture,11436,Undeclared,1,5,5,1,1,1,0
2,PHYS,000E987A7E3328E,0,WHITE,0,0,1,1,NA,0,⋯,Lecture,11243,No Subplan,0,NA,0,0,1,1,0
3,PHYS,004325A54F42C34,0,WHITE,0,0,0,0,113456,0,⋯,Lecture,11259,Pre-Physical Therapy,1,3,3,0,1,1,0
4,PHYS,0049F8A318A0EFD,0,WHITE,0,0,0,0,NA,0,⋯,Lecture,11543,Undeclared,0,NA,0,0,1,1,0
5,PHYS,005AF6C05F01E92,0,HISPA,1,1,1,1,98615,0,⋯,Lecture,11243,Computer Engineering,0,NA,0,0,1,1,0
6,PHYS,0078BB844C9CF20,0,WHITE,0,0,0,0,255081,0,⋯,Lecture,11496,Mechanical Engineering,0,NA,0,0,1,1,0


In [0]:
#### Create Stacked Dataset #### 

#Stacked data
df_clean <- rbind(df_bio, df_chem, df_phys)
df_clean <- df_clean %>%
  rename_at(vars(ends_with(".x")), 
            ~(str_replace(., ".x", "_2"))) %>%
  rename_at(vars(ends_with(".y")), 
            ~(str_replace(., ".y", "")))

rm(df_ap_bio, df_ap_chem, df_ap_phys, 
   df_bio, df_chem, df_phys, 
   df_crs_bio1, df_crs_bio2, df_crs_chem1, df_crs_chem2, df_crs_phys1, df_crs_phys2,
   df_crs, df_std)

names(df_clean)
head(df_clean)

# write.csv(df_clean, file = "SEISMIC_AP_CLEAN.csv")

[1] "course"             "st_id"              "firstgen"          
 [4] "ethniccode"         "ethniccode_cat"     "urm"               
 [7] "gender"             "female"             "famincome"         
[10] "lowincomflag"       "transfer"           "international"     
[13] "ell"                "us_hs"              "cohort"            
[16] "cohort_2013"        "cohort_2014"        "cohort_2015"       
[19] "cohort_2016"        "cohort_2017"        "cohort_2018"       
[22] "apyear"             "englsr"             "mathsr"            
[25] "hsgpa"              "crs_sbj_2"          "crs_catalog_2"     
[28] "crs_name_2"         "numgrade_2"         "numgrade_w_2"      
[31] "crs_retake_2"       "crs_term_2"         "summer_crs_2"      
[34] "TERM_REF_2"         "enrl_from_cohort_2" "crs_credits_2"     
[37] "crs_component_2"    "class_number_2"     "current_major_2"   
[40] "crs_sbj"            "crs_catalog"        "crs_name"          
[43] "numgrade"           "numgrade_w"         "crs_retake"        
[46] "crs_term"           "summer_crs"         "TERM_REF"          
[49] "enrl_from_cohort"   "crs_credits"        "crs_component"     
[52] "class_number"       "current_major"      "aptaker"           
[55] "apscore"            "apscore_full"       "apskipper"         
[58] "tookcourse"         "tookcourse_2"       "skipped_1"

,course,st_id,firstgen,ethniccode,ethniccode_cat,urm,gender,female,famincome,lowincomflag,⋯,crs_component,class_number,current_major,aptaker,apscore,apscore_full,apskipper,tookcourse,tookcourse_2,skipped_1
,<chr>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,⋯,<fct>,<int>,<fct>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,BIO,006193FE8B693CE,0,WHITE,0,0,0,0,NA,0,⋯,Lecture,10889,Biological Sciences,0,NA,0,0,1,1,0
2,BIO,00767AA686DED2D,0,WHITE,0,0,0,0,153056,0,⋯,Lecture,29164,Pre-Medicine,0,NA,0,0,1,1,0
3,BIO,00806194870DD09,0,WHITE,0,0,0,0,84630,0,⋯,Lecture,10197,Biological Sciences,0,NA,0,0,1,1,0
4,BIO,008593BB28EA023,1,ASIAN,2,0,0,0,21782,1,⋯,Lecture,10199,Pharmacy,1,3,3,0,1,1,0
5,BIO,00915F37523B8A2,1,WHITE,0,0,1,1,107815,0,⋯,Lecture,29164,Biological Sciences,1,2,2,0,1,1,0
6,BIO,00A397685C01F09,0,WHITE,0,0,1,1,105930,0,⋯,Lecture,10195,Pre-Medicine,1,3,3,0,1,1,0
